In [1]:
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
import warnings
import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

warnings.filterwarnings("ignore")
df = pd.read_excel('/kaggle/input/breast-cancer-wisconsin/cancer.xlsx')

In [2]:
from sklearn.preprocessing import MinMaxScaler
data = df.rename(columns={'Column1': 'ID', 'Column2': 'Y', 'Column3': 'radius1', 'Column4': 'texture1', 'Column5': 'perimeter1', 
                     'Column6': 'area1', 'Column7': 'smoothness1', 'Column8': 'compactness1', 'Column9': 'concavity1', 'Column10': 'concave_points1',
                     'Column11': 'symmetry1', 'Column12': 'fractal_dimension1', 'Column13': 'radius2', 'Column14': 'texture2', 'Column15': 'perimeter2', 
                     'Column16': 'area2', 'Column17': 'smoothness2', 'Column18': 'compactness2', 'Column19': 'concavity2', 'Column20': 'concave_points2',
                     'Column21': 'symmetry2', 'Column22': 'fractal_dimension2', 'Column23': 'radius3', 'Column24': 'texture3', 'Column25': 'perimeter3', 
                     'Column26': 'area3', 'Column27':'smoothness3', 'Column28': 'compactness3', 'Column29': 'concavity3', 'Column30': 'concave_points3',
                     'Column31': 'symmetry3', 'Column32': 'fractal_dimension3'})
data['Y'] = data['Y'].replace({'B': 0, 'M': 1})
# features_to_normalize = ['var1', 'var2', 'var3', 'var4', 'var5', 'var6', 'var7', 'var8', 'var9', 'var0']
cols_to_normalize = data.columns[2:]

# Normalize the selected columns using Min-Max scaling
scaler = MinMaxScaler()
data[cols_to_normalize] = scaler.fit_transform(data[cols_to_normalize])
#column_names = data.columns.tolist()
data

,ID,Y,radius1,texture1,perimeter1,area1,smoothness1,compactness1,concavity1,concave_points1,...,radius3,texture3,perimeter3,area3,smoothness3,compactness3,concavity3,concave_points3,symmetry3,fractal_dimension3
0,842302,1,0.521037,0.022658,0.545989,0.363733,0.593753,0.792037,0.703140,0.731113,...,0.620776,0.141525,0.668310,0.450698,0.601136,0.619292,0.568610,0.912027,0.598462,0.418864
1,842517,1,0.643144,0.272574,0.615783,0.501591,0.289880,0.181768,0.203608,0.348757,...,0.606901,0.303571,0.539818,0.435214,0.347553,0.154563,0.192971,0.639175,0.233590,0.222878
2,84300903,1,0.601496,0.390260,0.595743,0.449417,0.514309,0.431017,0.462512,0.635686,...,0.556386,0.360075,0.508442,0.374508,0.483590,0.385375,0.359744,0.835052,0.403706,0.213433
3,84348301,1,0.210090,0.360839,0.233501,0.102906,0.811321,0.811361,0.565604,0.522863,...,0.248310,0.385928,0.241347,0.094008,0.915472,0.814012,0.548642,0.884880,1.000000,0.773711
4,84358402,1,0.629893,0.156578,0.630986,0.489290,0.430351,0.347893,0.463918,0.518390,...,0.519744,0.123934,0.506948,0.341575,0.437364,0.172415,0.319489,0.558419,0.157500,0.142595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,926424,1,0.690000,0.428813,0.678668,0.566490,0.526948,0.296055,0.571462,0.690358,...,0.623266,0.383262,0.576174,0.452664,0.461137,0.178527,0.328035,0.761512,0.097575,0.105667
565,926682,1,0.622320,0.626987,0.604036,0.474019,0.407782,0.257714,0.337395,0.486630,...,0.560655,0.699094,0.520892,0.379915,0.300007,0.159997,0.256789,0.559450,0.198502,0.074315
566,926954,1,0.455251,0.621238,0.445788,0.303118,0.288165,0.254340,0.216753,0.263519,...,0.393099,0.589019,0.379949,0.230731,0.282177,0.273705,0.271805,0.487285,0.128721,0.151909
567,927241,1,0.644564,0.663510,0.665538,0.475716,0.588336,0.790197,0.823336,0.755467,...,0.633582,0.730277,0.668310,0.402035,0.619626,0.815758,0.749760,0.910653,0.497142,0.452315


In [3]:
def normal_prediction(X_train,y_train,X_test,y_test,input_s):
    input_dim = input_s
    running_times = []
    acc= []
    for i in range(0,10,1):
        model = keras.Sequential()
        model.add(layers.Dense(10, input_shape=(input_dim,)))

        model.add(layers.Dense(1, activation='sigmoid'))
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        start_time = time.time()
        model.fit(X_train,y_train, epochs=250, batch_size=32, verbose=0)
        predictions = model.predict(X_test)
        predicted = (predictions > 0.5)  
        accuracy = accuracy_score(y_test, predicted) 
       
        end_time = time.time()
        run_time = end_time - start_time
        running_times.append(run_time)
        acc.append(accuracy)
    avg_running_time = np.mean(running_times)
    avg_accuracy = np.mean(acc)

    print("Average running time:", avg_running_time, "s")
    print("Average accuracy:", avg_accuracy * 100, "%")

In [4]:
def drop_out_prediction(X_train,y_train,X_test,y_test,input_s):
    input_dim = input_s
    running_times = []
    acc= []
    for i in range(0,10,1):
        model = keras.Sequential()
        model.add(layers.Dense(10, activation='relu', input_shape=(input_dim,)))
        model.add(layers.Dropout(0.3))  # Adding dropout with rate 0.5

        model.add(layers.Dense(1, activation='sigmoid'))

        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

        start_time = time.time()
        model.fit(X_train, y_train, epochs=250, batch_size=32, verbose=0)
        predictions = model.predict(X_test)
        predicted = (predictions > 0.5)
        accuracy = accuracy_score(y_test, predicted)  
        # 输出准确率
        end_time = time.time()
        run_time = end_time - start_time
        running_times.append(run_time)
        acc.append(accuracy)
    avg_running_time = np.mean(running_times)
    avg_accuracy = np.mean(acc)

    print("Average running time:", avg_running_time, "s")
    print("Average accuracy:", avg_accuracy * 100, "%")  

# Sinle layer

In [5]:
var_t = data.loc[:, ['concavity1','radius3', 'concavity3', 'fractal_dimension3','Y']]

X_train, X_test, y_train, y_test = train_test_split(var_t[['concavity1','radius3', 'concavity3', 'fractal_dimension3']], var_t['Y'], test_size=0.2, random_state=42)
input_data = []
output_data = []
input_test =  []
output_test = []
for i in X_train.values:
    input_data.append(i)
for i in y_train.values:  
    output_data.append(i)
for i in X_test.values:
    input_test.append(i)
for i in y_test.values:
    output_test.append(i)

In [6]:
normal_prediction(X_train,y_train,X_test,y_test,np.shape(var_t)[1]-1)
drop_out_prediction(X_train,y_train,X_test,y_test,np.shape(var_t)[1]-1)

4/4 [==============================] - 0s 2ms/step
Average running time: 6.583042287826538 s
Average accuracy: 96.66666666666669 %
4/4 [==============================] - 0s 2ms/step
Average running time: 7.426801300048828 s
Average accuracy: 97.10526315789473 %


# Multiple layer

In [7]:
var_t = data.loc[:, ['concavity1','concave_points1','radius3','texture3','concavity3', 'symmetry3','Y']]

X_train, X_test, y_train, y_test = train_test_split(var_t[['concavity1','concave_points1','radius3','texture3','concavity3', 'symmetry3']], var_t['Y'], test_size=0.2, random_state=42)
input_data = []
output_data = []
input_test =  []
output_test = []
for i in X_train.values:
    input_data.append(i)
for i in y_train.values:  
    output_data.append(i)
for i in X_test.values:
    input_test.append(i)
for i in y_test.values:
    output_test.append(i)

In [8]:
normal_prediction(X_train,y_train,X_test,y_test,np.shape(var_t)[1]-1)
drop_out_prediction(X_train,y_train,X_test,y_test,np.shape(var_t)[1]-1)

4/4 [==============================] - 0s 2ms/step
Average running time: 8.136424112319947 s
Average accuracy: 94.73684210526315 %
4/4 [==============================] - 0s 2ms/step
Average running time: 9.408484387397767 s
Average accuracy: 95.08771929824562 %


# All variable

In [9]:
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:,2:], data['Y'], test_size=0.2, random_state=42)
input_data = []
output_data = []
input_test =  []
output_test = []
for i in X_train.values:
    input_data.append(i)
for i in y_train.values:  
    output_data.append(i)
for i in X_test.values:
    input_test.append(i)
for i in y_test.values:
    output_test.append(i)

In [10]:
normal_prediction(X_train,y_train,X_test,y_test,30)
drop_out_prediction(X_train,y_train,X_test,y_test,30)

4/4 [==============================] - 0s 2ms/step
Average running time: 7.198666763305664 s
Average accuracy: 97.98245614035088 %
4/4 [==============================] - 0s 2ms/step
Average running time: 8.53656725883484 s
Average accuracy: 98.15789473684208 %


# VWA one layer

In [11]:
var_t = data.loc[:, ['texture3','concavity3','symmetry3','Y']]

X_train, X_test, y_train, y_test = train_test_split(var_t[['texture3','concavity3','symmetry3']], var_t['Y'], test_size=0.2, random_state=42)
input_data = []
output_data = []
input_test =  []
output_test = []
for i in X_train.values:
    input_data.append(i)
for i in y_train.values:  
    output_data.append(i)
for i in X_test.values:
    input_test.append(i)
for i in y_test.values:
    output_test.append(i)

In [12]:
normal_prediction(X_train,y_train,X_test,y_test,np.shape(var_t)[1]-1)
drop_out_prediction(X_train,y_train,X_test,y_test,np.shape(var_t)[1]-1)

4/4 [==============================] - 0s 2ms/step
Average running time: 7.543458127975464 s
Average accuracy: 83.24561403508773 %
4/4 [==============================] - 0s 2ms/step
Average running time: 8.085574841499328 s
Average accuracy: 85.26315789473684 %


# VWA multiple layers

In [13]:
var_t = data.loc[:, ['radius1','texture1','area1','compactness1','concavity1','concave_points1','texture2','area3','concave_points3','area2','symmetry3','smoothness3','Y']]

X_train, X_test, y_train, y_test = train_test_split(var_t[['radius1','texture1','area1','compactness1','concavity1','concave_points1','texture2','area3','concave_points3','area2','symmetry3','smoothness3']], var_t['Y'], test_size=0.2, random_state=42)
input_data = []
output_data = []
input_test =  []
output_test = []
for i in X_train.values:
    input_data.append(i)
for i in y_train.values:  
    output_data.append(i)
for i in X_test.values:
    input_test.append(i)
for i in y_test.values:
    output_test.append(i)
    

In [14]:
normal_prediction(X_train,y_train,X_test,y_test,np.shape(var_t)[1]-1)
drop_out_prediction(X_train,y_train,X_test,y_test,np.shape(var_t)[1]-1)

4/4 [==============================] - 0s 2ms/step
Average running time: 7.614781832695007 s
Average accuracy: 98.2456140350877 %
4/4 [==============================] - 0s 3ms/step
Average running time: 8.33725848197937 s
Average accuracy: 97.71929824561403 %


# VWA one & multiple layers

In [15]:
var_t = data.loc[:, ['radius1', 'texture1', 'area1', 'compactness1', 'concave_points1', 'symmetry1', 'texture2', 
                     'area2', 'symmetry2', 'fractal_dimension2', 'radius3', 'texture3', 'area3', 'compactness3', 'concavity3', 'concave_points3','Y']]

X_train, X_test, y_train, y_test = train_test_split(var_t[['radius1', 'texture1', 'area1', 'compactness1', 'concave_points1', 'symmetry1', 'texture2', 'area2', 
                                                           'symmetry2', 'fractal_dimension2', 'radius3', 'texture3', 'area3', 'compactness3', 'concavity3', 'concave_points3']],
                                                    var_t['Y'], test_size=0.2, random_state=42)
# X_train = (X_train.values).reshape(-1,1)
# X_test = (X_test.values).reshape(-1, 1)
input_data = []
output_data = [] 
input_test =  []
output_test = []
for i in X_train.values:
    input_data.append(i)
for i in y_train.values:  
    output_data.append(i)
for i in X_test.values:
    input_test.append(i)
for i in y_test.values:
    output_test.append(i)

In [16]:
normal_prediction(X_train,y_train,X_test,y_test,np.shape(var_t)[1]-1)
drop_out_prediction(X_train,y_train,X_test,y_test,np.shape(var_t)[1]-1)

4/4 [==============================] - 0s 2ms/step
Average running time: 7.552580976486206 s
Average accuracy: 97.36842105263158 %
4/4 [==============================] - 0s 2ms/step
Average running time: 8.162972688674927 s
Average accuracy: 96.57894736842107 %


## Simplification

In [17]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
import warnings
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler
import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tqdm import tqdm
warnings.filterwarnings("ignore")

df = pd.read_excel('/kaggle/input/breast-cancer-wisconsin/cancer.xlsx')
data = df.rename(columns={'Column1': 'ID', 'Column2': 'Y', 'Column3': 'radius1', 'Column4': 'texture1', 'Column5': 'perimeter1', 
                     'Column6': 'area1', 'Column7': 'smoothness1', 'Column8': 'compactness1', 'Column9': 'concavity1', 'Column10': 'concave_points1',
                     'Column11': 'symmetry1', 'Column12': 'fractal_dimension1', 'Column13': 'radius2', 'Column14': 'texture2', 'Column15': 'perimeter2', 
                     'Column16': 'area2', 'Column17': 'smoothness2', 'Column18': 'compactness2', 'Column19': 'concavity2', 'Column20': 'concave_points2',
                     'Column21': 'symmetry2', 'Column22': 'fractal_dimension2', 'Column23': 'radius3', 'Column24': 'texture3', 'Column25': 'perimeter3', 
                     'Column26': 'area3', 'Column27':'smoothness3', 'Column28': 'compactness3', 'Column29': 'concavity3', 'Column30': 'concave_points3',
                     'Column31': 'symmetry3', 'Column32': 'fractal_dimension3'})
data['Y'] = data['Y'].replace({'B': 0, 'M': 1})
# features_to_normalize = ['var1', 'var2', 'var3', 'var4', 'var5', 'var6', 'var7', 'var8', 'var9', 'var0']
cols_to_normalize = data.columns[2:]

# Normalize the selected columns using Min-Max scaling
scaler = MinMaxScaler()
data[cols_to_normalize] = scaler.fit_transform(data[cols_to_normalize])


def reduce_weight(lambda_array1,lambda_array2,value,variables):
    lambda_array2 = np.where(lambda_array2 == 0, 2, lambda_array2)
#     print(lambda_array2)
    layer1_weight = lambda_array1
    layer2_weight = lambda_array2
    layer1_weight = np.where(lambda_array1[:30, :] < value, 0, 1)
    layer1_weight = layer1_weight[variables, :]
    layer2_weight = np.where(lambda_array2[:10, :] < value, 0, 1)
    
    zero_count = 10*len(selected_vars1)+ 10 - np.count_nonzero(layer1_weight)-np.count_nonzero(layer2_weight)
    rate = zero_count/(10*len(variables)+ 10)
    return rate,layer1_weight,layer2_weight


def reduce_weight_prediction(X_train,y_train,X_test,y_test,input_s, weight1, weight2):
    # Assuming you have your custom weights for the first layer
    input_dim = input_s
    running_times = []
    acc = []
    
    custom_first_layer_weights = weight1  
    custom_second_layer_weights = weight2  

    for i in range(0, 10, 1):
        model = keras.Sequential()
        model.add(Dense(10, activation='relu', input_shape=(input_dim,), weights=[custom_first_layer_weights, np.zeros(10)]))
        model.add(Dense(1, activation='sigmoid', weights=[custom_second_layer_weights, np.zeros(1)]))
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

        start_time = time.time()+1
        model.fit(X_train, y_train, epochs=250, batch_size=32, verbose=0)
        predictions = model.predict(X_test)
        predicted = (predictions > 0.5)
        accuracy = accuracy_score(y_test, predicted)

        end_time = time.time()
        run_time = end_time - start_time
        running_times.append(run_time)
        acc.append(accuracy)

    avg_running_time = np.mean(running_times)
    avg_accuracy = np.mean(acc)

    print("Average running time:", avg_running_time, "s")
    print("Average accuracy:", avg_accuracy * 100, "%")
import numpy as np
from scipy.sparse.linalg import eigsh

## some useful functions
def get_arccos(X):
    # X is a 2-d array
    
    n, p = X.shape
    cos_a = np.zeros([n, n, n])
    
    for r in range(n):
        
        xr = X[r]
        X_r = X - xr
        cross = np.dot(X_r, X_r.T)
        row_norm = np.sqrt(np.sum(X_r**2, axis = 1))
        outer_norm = np.outer(row_norm, row_norm)
        
        zero_idx = (outer_norm == 0.)
        outer_norm[zero_idx] = 1.
        cos_a_kl = cross / outer_norm
        cos_a_kl[zero_idx] = 0.

        cos_a[:,:,r] = cos_a_kl
        
    cos_a[cos_a > 1] = 1.
    cos_a[cos_a < -1] = -1.
    a = np.arccos(cos_a)

    a_bar_12 = np.mean(a, axis = 0, keepdims = True)
    a_bar_02 = np.mean(a, axis = 1, keepdims = True)
    a_bar_2  = np.mean(a, axis = (0,1), keepdims = True)
    A = a - a_bar_12 - a_bar_02 + a_bar_2
        
    return a, A

def get_arccos_1d(X):
    # X is a 1-d array
    
    X = np.squeeze(X)
    Y = X[:,None] - X
    Z = Y.T[:,:,None]*Y.T[:,None]
    n = len(X)
    
    a = np.zeros([n, n, n])
    a[Z == 0.] = np.pi/2.
    a[Z < 0.] = np.pi
    
    a = np.transpose(a, (1,2,0))
    
    #a = Z[Z>0.]*0. + Z[Z==0.]*np.pi/2. + Z[Z<0.]*np.pi

    a_bar_12 = np.mean(a, axis = 0, keepdims = True)
    a_bar_02 = np.mean(a, axis = 1, keepdims = True)
    a_bar_2  = np.mean(a, axis = (0,1), keepdims = True)
    A = a - a_bar_12 - a_bar_02 + a_bar_2
    
    return a, A

def orthonormalize(X):
    # X is a 2-d array
    # output: Gram-Schmidt orthogonalization of X
    
    n, p = X.shape
    Y = np.zeros([n,p])
    Y[:,0] = X[:,0]/np.sqrt(np.sum(X[:,0]**2))
    
    for j in range(1,p):
        
        Yj = Y[:,range(j)]
        xj = X[:,j]
        w = np.dot(xj, Yj)
        xj_p = np.sum(w*Yj, axis = 1)
        yj = xj - xj_p
        yj = yj/np.sqrt(np.sum(yj**2))
        
        Y[:,j] = yj
        
    return Y

# Main functions
def projection_corr(X, Y):
    # X, Y are 2-d array
    
    nx, p = X.shape
    ny, q = Y.shape
    
    if nx == ny:
        n = nx
    else:
        raise ValueError("sample sizes do not match.")
        
    a_x, A_x = get_arccos(X)
    a_y, A_y = get_arccos(Y)
    
    S_xy = np.sum(A_x * A_y) / (n**3)
    S_xx = np.sum(A_x**2) / (n**3)
    S_yy = np.sum(A_y**2) / (n**3)
    
    if S_xx * S_yy == 0.:
        corr = 0.
    else:
        corr = np.sqrt( S_xy / np.sqrt(S_xx * S_yy) )
    
    return corr

def projection_corr_1d(X, Y):
    
    nx, p = X.shape
    ny, q = Y.shape
    
    if nx == ny:
        n = nx
    else:
        raise ValueError("sample sizes do not match.")
        
    a_x, A_x = get_arccos_1d(X)
    a_y, A_y = get_arccos_1d(Y)
    
    S_xy = np.sum(A_x * A_y) / (n**3)
    S_xx = np.sum(A_x**2) / (n**3)
    S_yy = np.sum(A_y**2) / (n**3)
    
    if S_xx * S_yy == 0.:
        corr = 0.
    else:
        corr = np.sqrt( S_xy / np.sqrt(S_xx * S_yy) )
    
    return corr

def projection_corr_1dy(X, Y):
    
    nx, p = X.shape
    ny, q = Y.shape
    
    if nx == ny:
        n = nx
    else:
        raise ValueError("sample sizes do not match.")
        
    a_x, A_x = get_arccos(X)
    a_y, A_y = get_arccos_1d(Y)
    
    S_xy = np.sum(A_x * A_y) / (n**3)
    S_xx = np.sum(A_x**2) / (n**3)
    S_yy = np.sum(A_y**2) / (n**3)
    
    if S_xx * S_yy == 0.:
        corr = 0.
    else:
        corr = np.sqrt( S_xy / np.sqrt(S_xx * S_yy) )
    
    return corr

def get_equi_features(X):
    # X is 2-d array
    
    n, p = X.shape
    scale = np.sqrt(np.sum(X**2, axis=0))
    Xstd = X / scale
    sigma = np.dot(Xstd.T, Xstd)
    sigma_inv = np.linalg.inv(sigma)
    lambd_min = eigsh(sigma, k=1, which='SA')[0].squeeze()
    sj = np.min([1., 2.*lambd_min])
    sj = sj - 0.00001
    
    mat_s = np.diag([sj]*p)
    A = 2*mat_s - sj*sj*sigma_inv
    C = np.linalg.cholesky(A).T
    
    Xn = np.random.randn(n, p)
    XX = np.hstack([Xstd, Xn])
    XXo = orthonormalize(XX)
    U = XXo[:,range(p,2*p)]
    
    Xnew = np.dot(Xstd,  np.eye(p) - sigma_inv*sj) + np.dot(U,C)
    return Xnew

X = data.iloc[:,2:]
Y = data.iloc[:,1]
X = (X - X.mean()) / X.std()
X_knockoff = get_equi_features(X)

X_knockoff_df = pd.DataFrame(X_knockoff)
column_names = ['radius1', 'texture1', 'perimeter1', 
                'area1', 'smoothness1', 'compactness1', 'concavity1', 'concave_points1',
                'symmetry1', 'fractal_dimension1', 'radius2', 'texture2', 'perimeter2', 
                'area2', 'smoothness2', 'compactness2', 'concavity2', 'concave_points2',
                'symmetry2', 'fractal_dimension2', 'radius3', 'texture3', 'perimeter3', 
                'area3', 'smoothness3', 'compactness3', 'concavity3', 'concave_points3',
                'symmetry3', 'fractal_dimension3']
X_knockoff_df.columns = column_names
feature = pd.concat([X,X_knockoff_df],axis = 1)
dataset1 =  pd.concat([feature,data['Y']],axis = 1)
dataset1
# dataset1.to_csv('dataset1_for_DeepPINK.csv', index=False)

lambda_array = np.zeros((60, 10))
lambda_array2 = np.zeros((10, 1))
input_dim = 60

# Use tqdm for a one-line progress bar
for i in tqdm(np.arange(0, 1, 0.01)):
    model = keras.Sequential()
    model.add(layers.Dense(10, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(i)))
    #model.add(layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(i)))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(feature, Y, epochs=50, batch_size=32, verbose=0)

    first_layer = model.layers[0]
    weights, biases = first_layer.get_weights()
    
    for j in range(min(len(weights), len(lambda_array))):
        for n in range(min(len(weights[j]), len(lambda_array[j]))):
            if abs(weights[j][n]) < 5e-4 and lambda_array[j][n] == 0:
                lambda_array[j][n] = i
    
    # Calculate the Zi in second layer
    second_layer = model.layers[1]
    weights, biases = second_layer.get_weights()
    #print(f"Layer: {layer.name}, Weights: {weights}")
#     print(weights)
#     for j in range(len(weights)):
#         for n in range(len(weights[j])):
#             if abs(weights[j][n]) < 5e-4 and lambda_array2[j][n] == 0 :
# #                 print(j,n)
#                 lambda_array2[j][n] = i
print(lambda_array)

100%|██████████| 100/100 [04:06<00:00,  2.46s/it]

[[0.17 0.28 0.08 0.1  0.01 0.06 0.23 0.19 0.05 0.19]
 [0.17 0.28 0.08 0.17 0.01 0.06 0.23 0.19 0.05 0.19]
 [0.17 0.28 0.11 0.1  0.01 0.06 0.23 0.19 0.05 0.21]
 [0.17 0.28 0.08 0.1  0.01 0.06 0.23 0.19 0.05 0.19]
 [0.01 0.28 0.08 0.17 0.01 0.06 0.23 0.19 0.05 0.19]
 [0.09 0.09 0.08 0.17 0.01 0.06 0.35 0.19 0.02 0.03]
 [0.09 0.28 0.08 0.1  0.01 0.2  0.23 0.19 0.05 0.19]
 [0.09 0.28 0.08 0.1  0.01 0.06 0.23 0.35 0.05 0.19]
 [0.17 0.28 0.08 0.1  0.01 0.06 0.23 0.19 0.05 0.19]
 [0.17 0.28 0.08 0.1  0.01 0.06 0.23 0.19 0.05 0.19]
 [0.09 0.28 0.08 0.1  0.01 0.06 0.23 0.19 0.05 0.19]
 [0.09 0.06 0.08 0.08 0.13 0.06 0.22 0.13 0.05 0.19]
 [0.09 0.28 0.08 0.1  0.01 0.06 0.23 0.19 0.05 0.19]
 [0.09 0.28 0.08 0.1  0.01 0.06 0.23 0.19 0.05 0.19]
 [0.15 0.15 0.08 0.08 0.13 0.06 0.16 0.15 0.05 0.1 ]
 [0.17 0.28 0.08 0.1  0.01 0.06 0.23 0.19 0.05 0.19]
 [0.17 0.02 0.08 0.05 0.01 0.06 0.23 0.19 0.05 0.09]
 [0.17 0.28 0.08 0.1  0.01 0.06 0.23 0.19 0.05 0.19]
 [0.09 0.28 0.08 0.1  0.01 0.06 0.23 0.13 0.05

### OL

In [66]:
selected_vars1 = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]
r,l1,l2 = reduce_weight(lambda_array,lambda_array2,0,selected_vars1)

In [67]:
from sklearn.model_selection import train_test_split
var_t = data.loc[:, ['radius1', 'texture1', 'perimeter1', 
                'area1', 'smoothness1', 'compactness1', 'concavity1', 'concave_points1',
                'symmetry1', 'fractal_dimension1', 'radius2', 'texture2', 'perimeter2', 
                'area2', 'smoothness2', 'compactness2', 'concavity2', 'concave_points2',
                'symmetry2', 'fractal_dimension2', 'radius3', 'texture3', 'perimeter3', 
                'area3', 'smoothness3', 'compactness3', 'concavity3', 'concave_points3',
                'symmetry3', 'fractal_dimension3','Y']]

X_train, X_test, y_train, y_test = train_test_split(var_t[['radius1', 'texture1', 'perimeter1', 
                'area1', 'smoothness1', 'compactness1', 'concavity1', 'concave_points1',
                'symmetry1', 'fractal_dimension1', 'radius2', 'texture2', 'perimeter2', 
                'area2', 'smoothness2', 'compactness2', 'concavity2', 'concave_points2',
                'symmetry2', 'fractal_dimension2', 'radius3', 'texture3', 'perimeter3', 
                'area3', 'smoothness3', 'compactness3', 'concavity3', 'concave_points3',
                'symmetry3', 'fractal_dimension3']], var_t['Y'], test_size=0.2, random_state=42)
input_data = []
output_data = []
input_test =  []
output_test = []
for i in X_train.values:
    input_data.append(i)
for i in y_train.values:  
    output_data.append(i)
for i in X_test.values:
    input_test.append(i)
for i in y_test.values:
    output_test.append(i)

In [68]:
reduce_weight_prediction(X_train,y_train,X_test,y_test,30, l1, l2)

4/4 [==============================] - 0s 2ms/step
Average running time: 8.102488684654237 s
Average accuracy: 97.98245614035088 %


### ML

In [69]:
var_t = data.loc[:, ['radius3','texture3','symmetry3','Y']]

X_train, X_test, y_train, y_test = train_test_split(var_t[['radius3','texture3','symmetry3']], var_t['Y'], test_size=0.2, random_state=42)
input_data = []
output_data = []
input_test =  []
output_test = []
for i in X_train.values:
    input_data.append(i)
for i in y_train.values:  
    output_data.append(i)
for i in X_test.values:
    input_test.append(i)
for i in y_test.values:
    output_test.append(i)

In [70]:
selected_vars1 = [20,21,28]
r,l1,l2 = reduce_weight(lambda_array,lambda_array2,0,selected_vars1)

In [71]:
reduce_weight_prediction(X_train,y_train,X_test,y_test,3, l1, l2)

4/4 [==============================] - 0s 2ms/step
Average running time: 7.49768180847168 s
Average accuracy: 96.49122807017545 %


### VWA_OL

In [72]:
var_t = data.loc[:, ['texture3','concavity3','symmetry3','Y']]

X_train, X_test, y_train, y_test = train_test_split(var_t[['texture3','concavity3','symmetry3']], var_t['Y'], test_size=0.2, random_state=42)
input_data = []
output_data = []
input_test =  []
output_test = []
for i in X_train.values:
    input_data.append(i)
for i in y_train.values:  
    output_data.append(i)
for i in X_test.values:
    input_test.append(i)
for i in y_test.values:
    output_test.append(i)

In [73]:
selected_vars1 = [21,26,28]
r,l1,l2 = reduce_weight(lambda_array,lambda_array2,0,selected_vars1)

In [74]:
reduce_weight_prediction(X_train,y_train,X_test,y_test,3, l1, l2)

4/4 [==============================] - 0s 2ms/step
Average running time: 6.877101802825928 s
Average accuracy: 82.45614035087719 %


### VWA_ML

In [75]:
var_t = data.loc[:, ['radius1','texture1','area1','compactness1','concavity1','concave_points1','texture2','area3','concave_points3','area2','symmetry3','smoothness3','Y']]

X_train, X_test, y_train, y_test = train_test_split(var_t[['radius1','texture1','area1','compactness1','concavity1','concave_points1','texture2','area3','concave_points3','area2','symmetry3','smoothness3']], var_t['Y'], test_size=0.2, random_state=42)
input_data = []
output_data = []
input_test =  []
output_test = []
for i in X_train.values:
    input_data.append(i)
for i in y_train.values:  
    output_data.append(i)
for i in X_test.values:
    input_test.append(i)
for i in y_test.values:
    output_test.append(i)
    

In [76]:
selected_vars1 = [0,1,3,5,6,7,11,13,24,23,27,28]
r,l1,l2 = reduce_weight(lambda_array,lambda_array2,0,selected_vars1)

In [77]:
reduce_weight_prediction(X_train,y_train,X_test,y_test,12, l1, l2)

4/4 [==============================] - 0s 2ms/step
Average running time: 7.459563899040222 s
Average accuracy: 97.45614035087718 %


### VWA_OML

In [78]:
selected_vars1 = [0,1,3,5,7,8,11,13,20,21,22,23,24,26,27,28]
r,l1,l2 = reduce_weight(lambda_array,lambda_array2,0,selected_vars1)

In [79]:
var_t = data.loc[:, ['radius1', 'texture1', 'area1', 'compactness1', 'concave_points1', 'symmetry1', 'texture2', 
                     'area2', 'symmetry2', 'fractal_dimension2', 'radius3', 'texture3', 'area3', 'compactness3', 'concavity3', 'concave_points3','Y']]

X_train, X_test, y_train, y_test = train_test_split(var_t[['radius1', 'texture1', 'area1', 'compactness1', 'concave_points1', 'symmetry1', 'texture2', 'area2', 
                                                           'symmetry2', 'fractal_dimension2', 'radius3', 'texture3', 'area3', 'compactness3', 'concavity3', 'concave_points3']],
                                                    var_t['Y'], test_size=0.2, random_state=42)
# X_train = (X_train.values).reshape(-1,1)
# X_test = (X_test.values).reshape(-1, 1)
input_data = []
output_data = [] 
input_test =  []
output_test = []
for i in X_train.values:
    input_data.append(i)
for i in y_train.values:  
    output_data.append(i)
for i in X_test.values:
    input_test.append(i)
for i in y_test.values:
    output_test.append(i)

In [80]:
reduce_weight_prediction(X_train,y_train,X_test,y_test,16, l1, l2)

4/4 [==============================] - 0s 2ms/step
Average running time: 7.492507290840149 s
Average accuracy: 96.49122807017545 %


## DeepPINK

In [86]:
var_t = data.loc[:, ['radius3', 'concave_points3', 'Y']]

X_train, X_test, y_train, y_test = train_test_split(var_t[['radius3', 'concave_points3']],
                                                    var_t['Y'], test_size=0.2, random_state=42)
# X_train = (X_train.values).reshape(-1,1)
# X_test = (X_test.values).reshape(-1, 1)
input_data = []
output_data = [] 
input_test =  []
output_test = []
for i in X_train.values:
    input_data.append(i)
for i in y_train.values:  
    output_data.append(i)
for i in X_test.values:
    input_test.append(i)
for i in y_test.values:
    output_test.append(i)

In [87]:
normal_prediction(X_train,y_train,X_test,y_test,np.shape(var_t)[1]-1)

4/4 [==============================] - 0s 3ms/step
Average running time: 8.36764407157898 s
Average accuracy: 96.49122807017545 %


## DeepLINK

In [83]:
var_t = data.loc[:, ['radius1', 'texture1','compactness1','concavity1','concave_points1','concave_points2','perimeter3','area3','Y']]

X_train, X_test, y_train, y_test = train_test_split(var_t[['radius1', 'texture1','compactness1','concavity1','concave_points1','concave_points2','perimeter3','area3']],
                                                    var_t['Y'], test_size=0.2, random_state=42)
# X_train = (X_train.values).reshape(-1,1)
# X_test = (X_test.values).reshape(-1, 1)
input_data = []
output_data = [] 
input_test =  []
output_test = []
for i in X_train.values:
    input_data.append(i)
for i in y_train.values:  
    output_data.append(i)
for i in X_test.values:
    input_test.append(i)
for i in y_test.values:
    output_test.append(i)

In [85]:
normal_prediction(X_train,y_train,X_test,y_test,np.shape(var_t)[1]-1)

4/4 [==============================] - 0s 2ms/step
Average running time: 8.26431736946106 s
Average accuracy: 95.78947368421053 %
